In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import (
    make_scorer, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [2]:
# 读取 train_Data 文件
data = pd.read_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/final_data/combine_data_drop.parquet')
#application_test = pd.read_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/application_test.csv')

# 打印 DataFrame
data

,NAME_CONTRACT_TYPE,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,...,SK_ID_CURR_var_refused_last_5,SK_ID_CURR_var_revolving_loan_last_5,SK_ID_CURR_var_cash_loan_last_5,SK_ID_CURR_var_consumer_loan_last_5,SK_ID_CURR_var_credit_goods_ratio_last_5,SK_ID_CURR_var_application_credit_diff_last_5,SK_ID_CURR_var_application_credit_ratio_last_5,previous_application_number_of_prev_application,previous_application_fraction_of_refused_applications,TARGET
0,0,0.0,1.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.100186,0.097330,0.167730,0.171049,0.017306,4.953836e+09,0.069064,1.0,0.000000,1
1,0,1.0,0.0,0.0,0,0,0.0,1.0,0.0,0.0,...,0.000000,0.000000,0.333333,0.333333,0.007015,5.728765e+09,0.005324,3.0,0.000000,0
2,1,0.0,1.0,0.0,1,1,0.0,0.0,0.0,0.0,...,0.100186,0.097330,0.167730,0.171049,0.017306,4.953836e+09,0.069064,1.0,0.000000,0
3,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.200000,0.300000,0.300000,0.000000,0.050180,9.514831e+09,0.028940,9.0,0.111111,0
4,0,0.0,1.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.200000,0.200000,0.012347,6.140349e+08,0.007897,6.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.105144,0.106273,0.166300,0.170487,0.016400,4.470164e+09,0.065782,1.0,0.000000,2
48740,0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.250000,0.250000,0.012453,1.640626e+08,0.009809,4.0,0.000000,2
48741,0,1.0,0.0,0.0,1,1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.005002,1.459742e+08,0.006176,2.0,0.000000,2
48742,0,0.0,1.0,0.0,0,0,0.0,1.0,0.0,0.0,...,0.300000,0.000000,0.300000,0.300000,0.010553,5.010032e+08,0.007871,5.0,0.400000,2


In [3]:
X= data.drop(['TARGET'],axis=1)
y = data['TARGET']

In [5]:
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),  
    'recall': make_scorer(recall_score),  
    'f1': make_scorer(f1_score)  
}

In [4]:
def clean_column_names(df):
    # 定义正则表达式，匹配所有非法字符
    pattern = r'[^a-zA-Z0-9_]'  # 允许字母、数字和下划线

    # 使用 str.replace() 替换非法字符为下划线
    df.columns = df.columns.str.replace(pattern, '_', regex=True)

    return df

In [5]:
data = clean_column_names(data)
data.columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER_F', 'CODE_GENDER_M',
       'CODE_GENDER_XNA', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE_Children', 'NAME_TYPE_SUITE_Family',
       'NAME_TYPE_SUITE_Group_of_people', 'NAME_TYPE_SUITE_Other_A',
       ...
       'SK_ID_CURR_var_refused_last_5', 'SK_ID_CURR_var_revolving_loan_last_5',
       'SK_ID_CURR_var_cash_loan_last_5',
       'SK_ID_CURR_var_consumer_loan_last_5',
       'SK_ID_CURR_var_credit_goods_ratio_last_5',
       'SK_ID_CURR_var_application_credit_diff_last_5',
       'SK_ID_CURR_var_application_credit_ratio_last_5',
       'previous_application_number_of_prev_application',
       'previous_application_fraction_of_refused_applications', 'TARGET'],
      dtype='object', length=898)

In [6]:
train_data = data[data['TARGET']!=2]
test_data = data[data['TARGET']==2]

In [6]:
del data

In [7]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),  
    'recall': make_scorer(recall_score),  
    'f1': make_scorer(f1_score)  
}
X_train, X_test, y_train, y_test = train_test_split(train_data.drop('TARGET',axis =1 ), train_data['TARGET'], test_size=0.2, random_state=42)

classifier = lgb.LGBMClassifier(random_state=42, verbosity=-1)# 关闭日志输出
classifier.fit(X_train, y_train)

LGBMClassifier(random_state=42, verbosity=-1)

In [51]:
# 计算指标

from sklearn.metrics import roc_auc_score
#test_encoded = pd.get_dummies(application_test, columns=columns, drop_first=True)
y_pred_proba = classifier.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 计算 AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f'AUC: {auc}')


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = classifier.predict(X_test)

# 计算各项指标
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 输出结果
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

AUC: 0.9806569408319772
Accuracy: 0.9553747512712801
Precision: 0.9968352598367457
Recall: 0.9136525230371956
F1 Score: 0.9534330011074197


In [52]:
test_ori = pd.read_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/application_test.csv')
test_ori

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [54]:
test_data

,NAME_CONTRACT_TYPE,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group_of_people,NAME_TYPE_SUITE_Other_A,...,SK_ID_CURR_var_refused_last_5,SK_ID_CURR_var_revolving_loan_last_5,SK_ID_CURR_var_cash_loan_last_5,SK_ID_CURR_var_consumer_loan_last_5,SK_ID_CURR_var_credit_goods_ratio_last_5,SK_ID_CURR_var_application_credit_diff_last_5,SK_ID_CURR_var_application_credit_ratio_last_5,previous_application_number_of_prev_application,previous_application_fraction_of_refused_applications,TARGET
0,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.105144,0.106273,0.166300,0.170487,0.016400,4.470164e+09,0.065782,1.0,0.000,2
1,0,0.0,1.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.500000,0.500000,0.016400,9.963648e+06,0.065782,2.0,0.000,2
2,0,0.0,1.0,0.0,1,1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.333333,0.333333,0.010466,9.903134e+08,0.009347,4.0,0.000,2
3,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.000000,0.200000,0.200000,0.300000,0.003173,1.027807e+10,0.270941,5.0,0.000,2
4,0,0.0,1.0,0.0,1,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.500000,0.500000,0.000315,8.965495e+08,0.000192,2.0,0.000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.105144,0.106273,0.166300,0.170487,0.016400,4.470164e+09,0.065782,1.0,0.000,2
48740,0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.250000,0.250000,0.012453,1.640626e+08,0.009809,4.0,0.000,2
48741,0,1.0,0.0,0.0,1,1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.005002,1.459742e+08,0.006176,2.0,0.000,2
48742,0,0.0,1.0,0.0,0,0,0.0,1.0,0.0,0.0,...,0.300000,0.000000,0.300000,0.300000,0.010553,5.010032e+08,0.007871,5.0,0.400,2


In [55]:
y_preb_proba = classifier.predict_proba(test_data.drop('TARGET',axis =1))[:, 1]
#y_pred_proba = model_flit.predict_proba(test_data_flit)[:, 1]

In [56]:
y_preb_proba

array([0.56658023, 0.80190285, 0.68285477, ..., 0.0809301 , 0.29663076,
       0.65380387])

In [57]:
y_preb_proba = pd.DataFrame(y_preb_proba, columns=['TARGET'],index = test_ori['SK_ID_CURR'])
y_preb_proba.index.name = 'SK_ID_CURR'
y_preb_proba

,TARGET
SK_ID_CURR,
100001,0.566580
100005,0.801903
100013,0.682855
100028,0.194267
100038,0.917939
...,...
456221,0.914706
456222,0.972473
456223,0.080930


In [58]:
y_preb_proba.to_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/result/lgm_para.csv')

In [8]:
importance = classifier.feature_importances_

# 创建特征重要性数据框
feature_importance = pd.DataFrame({
    'Feature': train_data.drop('TARGET',axis=1).columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

# 打印特征重要性
print(feature_importance)

                                         Feature  Importance
212                     OBS_30_CNT_SOCIAL_CIRCLE          68
213                     DEF_30_CNT_SOCIAL_CIRCLE          63
648                 pos_cash_completed_contracts          62
584                        external_sources_mean          60
629  SK_ID_CURR_mean_instalment_paid_late_last_5          58
..                                           ...         ...
536               SK_ID_CURR_mean_MONTHS_BALANCE           0
534   SK_ID_CURR_mean_CNT_DRAWINGS_OTHER_CURRENT           0
40                OCCUPATION_TYPE_Cleaning_staff           0
13                           NAME_TYPE_SUITE_XNA           0
165                   REG_REGION_NOT_WORK_REGION           0

[897 rows x 2 columns]


In [9]:
# 筛选指标

feature_importance['cumulative_importance'] = feature_importance['Importance'].cumsum()

# 选择累计重要性达到80%的因子
selected_features = feature_importance[feature_importance['cumulative_importance'] <= 1600]

# 获取选择的因子
result = selected_features['Feature']

result

212                             OBS_30_CNT_SOCIAL_CIRCLE
213                             DEF_30_CNT_SOCIAL_CIRCLE
648                         pos_cash_completed_contracts
584                                external_sources_mean
629          SK_ID_CURR_mean_instalment_paid_late_last_5
                             ...                        
400    SK_ID_CURR_mean_bureau_credit_type_consumer_la...
276                    OCCUPATION_TYPE_mean_EXT_SOURCE_1
670                previous_application_prev_was_refused
665    SK_ID_CURR_mean_pos_cash_paid_late_with_tolera...
23                  NAME_EDUCATION_TYPE_Higher_education
Name: Feature, Length: 61, dtype: object

In [61]:
trian_data_flit = train_data[result]
test_data_flit = test_data[result]

In [62]:
trian_data_flit

,DEF_30_CNT_SOCIAL_CIRCLE,SK_ID_CURR_mean_instalment_paid_late_last_5,CNT_FAM_MEMBERS,external_sources_mean,credit_to_goods_ratio,phone_to_employ_ratio,pos_cash_completed_contracts,OBS_30_CNT_SOCIAL_CIRCLE,SK_ID_CURR_max_instalment_paid_late_last_5,external_sources_sum,...,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY,external_sources_nanmedian,payment_rate,SK_ID_CURR_mean_CNT_DRAWINGS_ATM_CURRENT,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH,credit_to_annuity_ratio,OCCUPATION_TYPE_mean_DAYS_EMPLOYED,SK_ID_CURR_max_CNT_PAYMENT_last_5,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN,previous_application_prev_was_refused
0,2.000000,0.000000,1.000000,0.161787,1.158397,1.780220,0.000000,2.000000,0.000000,0.485361,...,28889.263426,0.139376,0.060749,0.566241,-3049.716221,16.461104,-2424.143152,24.000000,0.442195,0.000000
1,0.000000,0.000000,2.000000,0.466757,1.145199,0.696970,2.000000,1.000000,0.000000,0.933513,...,26313.573918,0.466757,0.027598,0.566241,-3087.977649,36.234085,-2797.755967,12.000000,0.365735,0.000000
2,0.000000,0.000000,1.000000,0.642739,1.000000,3.622222,1.000000,0.000000,0.000000,1.285479,...,28889.263426,0.642739,0.050000,0.566241,-3049.716221,20.000000,-2424.143152,4.000000,0.442195,0.000000
3,0.000000,0.000000,2.000000,0.650442,1.052803,0.203027,2.000000,2.000000,0.000000,0.650442,...,26313.573918,0.650442,0.094941,0.566241,-3087.977649,10.532818,-2424.143152,48.000000,0.365735,1.000000
4,0.000000,0.000000,1.000000,0.322738,1.000000,0.364055,3.000000,0.000000,0.000000,0.322738,...,28144.327169,0.322738,0.042623,0.566241,-2870.258504,23.461618,-2797.755967,48.000000,0.514099,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565367,0.000000,0.319892,3.000000,0.442642,1.369532,0.848098,0.799729,1.401895,0.799729,0.807402,...,28364.133034,0.442642,0.048163,0.566241,-2958.015124,20.807239,-2279.877369,24.000000,0.455176,0.000000
565368,0.000000,0.398049,3.504878,0.463903,1.158271,441.761481,1.247561,0.752439,1.000000,1.299416,...,26313.573918,0.534672,0.029211,0.566241,-3087.977649,34.235456,-1869.701595,12.495122,0.365735,0.000000
565369,0.000000,0.000000,2.699124,0.510719,1.047659,0.987711,2.398248,0.000000,0.000000,0.571286,...,26313.573918,0.510719,0.052465,0.817099,-3087.977649,19.246774,-2395.049057,39.610513,0.365735,0.699124
565370,0.543924,0.217570,1.456076,0.264071,1.165803,0.353888,3.087848,0.543924,0.543924,0.702361,...,26313.573918,0.275082,0.073321,0.578204,-3087.977649,13.756022,-3298.318357,50.209372,0.365735,0.456076


In [22]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assume X_train and X_test are your training and testing datasets
# Standardizing the data before applying PCA is important
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(trian_data_flit)
X_test_scaled = scaler.transform(test_data_flit)

# Apply PCA on the training set
pca = PCA(n_components=0.80)  # Retain 95% of variance, can adjust n_components
X_train_pca = pca.fit_transform(X_train_scaled)

# Apply the same transformation to the test set using the training PCA transformation matrix
X_test_pca = pca.transform(X_test_scaled)

# Optional: converting back to a DataFrame for better interpretability
X_train_pca_df = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(X_train_pca.shape[1])],
                              index=trian_data_flit.index)
X_test_pca_df = pd.DataFrame(X_test_pca, columns=[f'PC{i+1}' for i in range(X_test_pca.shape[1])],
                             index=test_data_flit.index)

# Print the explained variance ratio to see how much variance is retained by each principal component
print("Explained variance ratio:", pca.explained_variance_ratio_)
print(X_train_pca_df)
print(X_test_pca_df)

Explained variance ratio: [0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667]
             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
0      -1.482471 -0.111557  0.573288 -0.530772  0.178061  0.172469  0.739262   
1      -0.061296 -0.234241 -0.113713  0.503727  0.080018 -0.381175 -0.500831   
2       1.281161  2.214455  1.166696  0.465819  1.185518  0.092317  0.055589   
3      -0.013611  0.011438  1.537688  0.458162  0.362583 -0.269062 -0.552199   
4      -0.146827  0.382016  0.559523  1.294810 -0.848342  1.010709  0.994443   
...          ...       ...       ...       ...       ...       ...       ...   
565367  0.524062 -1.225691  0.612574 -1.413175  1.140957 -1.482892 -2.058750   
565368 -0.276093 -0.574883 -0.930836  1.035438  0.548180 -1.205776  0.208098   
565369  0.362036 -0.446113  0

In [81]:
# 重新训练


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(trian_data_flit, train_data['TARGET'], test_size=0.2, random_state=42)\


# 创建XGBoost模型
model_flit = lgb.LGBMClassifier(use_label_encoder=False, eval_metric='logloss',learning_rate = 0.03, max_depth = 14, 
                                        num_leaves = 273, min_data_in_leaf = 100,feature_fraction = 0.6042460007414772,
                                        bagging_fraction = 0.8802727495826554, bagging_freq = 10, lambda_l1 = 22.70784666063976	,
                                        lambda_l2 = 40.733683783736836, min_gain_to_split = 11, n_estimators = 526)

# 拟合模型
model_flit.fit(X_train,y_train)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] lambda_l1 is set=22.70784666063976, reg_alpha=0.0 will be ignored. Current value: lambda_l1=22.70784666063976
[LightGBM] [Warning] lambda_l2 is set=40.733683783736836, reg_lambda=0.0 will be ignored. Current value: lambda_l2=40.733683783736836
[LightGBM] [Warning] bagging_fraction is set=0.8802727495826554, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8802727495826554
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_gain_to_split is set=11, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11
[LightGBM] [Warning] feature_fraction is set=0.6042460007414772, colsample_bytree=1.0 will be ignored. Current value: feat

LGBMClassifier(bagging_fraction=0.8802727495826554, bagging_freq=10,
               eval_metric='logloss', feature_fraction=0.6042460007414772,
               lambda_l1=22.70784666063976, lambda_l2=40.733683783736836,
               learning_rate=0.03, max_depth=14, min_data_in_leaf=100,
               min_gain_to_split=11, n_estimators=526, num_leaves=273,
               use_label_encoder=False)

In [85]:
trian_data_flit

,DEF_30_CNT_SOCIAL_CIRCLE,SK_ID_CURR_mean_instalment_paid_late_last_5,CNT_FAM_MEMBERS,external_sources_mean,credit_to_goods_ratio,phone_to_employ_ratio,pos_cash_completed_contracts,OBS_30_CNT_SOCIAL_CIRCLE,SK_ID_CURR_max_instalment_paid_late_last_5,external_sources_sum,...,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY,external_sources_nanmedian,payment_rate,SK_ID_CURR_mean_CNT_DRAWINGS_ATM_CURRENT,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH,credit_to_annuity_ratio,OCCUPATION_TYPE_mean_DAYS_EMPLOYED,SK_ID_CURR_max_CNT_PAYMENT_last_5,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN,previous_application_prev_was_refused
0,2.000000,0.000000,1.000000,0.161787,1.158397,1.780220,0.000000,2.000000,0.000000,0.485361,...,28889.263426,0.139376,0.060749,0.566241,-3049.716221,16.461104,-2424.143152,24.000000,0.442195,0.000000
1,0.000000,0.000000,2.000000,0.466757,1.145199,0.696970,2.000000,1.000000,0.000000,0.933513,...,26313.573918,0.466757,0.027598,0.566241,-3087.977649,36.234085,-2797.755967,12.000000,0.365735,0.000000
2,0.000000,0.000000,1.000000,0.642739,1.000000,3.622222,1.000000,0.000000,0.000000,1.285479,...,28889.263426,0.642739,0.050000,0.566241,-3049.716221,20.000000,-2424.143152,4.000000,0.442195,0.000000
3,0.000000,0.000000,2.000000,0.650442,1.052803,0.203027,2.000000,2.000000,0.000000,0.650442,...,26313.573918,0.650442,0.094941,0.566241,-3087.977649,10.532818,-2424.143152,48.000000,0.365735,1.000000
4,0.000000,0.000000,1.000000,0.322738,1.000000,0.364055,3.000000,0.000000,0.000000,0.322738,...,28144.327169,0.322738,0.042623,0.566241,-2870.258504,23.461618,-2797.755967,48.000000,0.514099,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565367,0.000000,0.319892,3.000000,0.442642,1.369532,0.848098,0.799729,1.401895,0.799729,0.807402,...,28364.133034,0.442642,0.048163,0.566241,-2958.015124,20.807239,-2279.877369,24.000000,0.455176,0.000000
565368,0.000000,0.398049,3.504878,0.463903,1.158271,441.761481,1.247561,0.752439,1.000000,1.299416,...,26313.573918,0.534672,0.029211,0.566241,-3087.977649,34.235456,-1869.701595,12.495122,0.365735,0.000000
565369,0.000000,0.000000,2.699124,0.510719,1.047659,0.987711,2.398248,0.000000,0.000000,0.571286,...,26313.573918,0.510719,0.052465,0.817099,-3087.977649,19.246774,-2395.049057,39.610513,0.365735,0.699124
565370,0.543924,0.217570,1.456076,0.264071,1.165803,0.353888,3.087848,0.543924,0.543924,0.702361,...,26313.573918,0.275082,0.073321,0.578204,-3087.977649,13.756022,-3298.318357,50.209372,0.365735,0.456076


In [82]:
# 计算指标

from sklearn.metrics import roc_auc_score
#test_encoded = pd.get_dummies(application_test, columns=columns, drop_first=True)
y_pred_proba = model_flit.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 计算 AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f'AUC: {auc}')


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model_flit.predict(X_test)

# 计算各项指标
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 输出结果
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] lambda_l1 is set=22.70784666063976, reg_alpha=0.0 will be ignored. Current value: lambda_l1=22.70784666063976
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] lambda_l2 is set=40.733683783736836, reg_lambda=0.0 will be ignored. Current value: lambda_l2=40.733683783736836
[LightGBM] [Warning] bagging_fraction is set=0.8802727495826554, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8802727495826554
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_gain_to_split is set=11, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11
[LightGBM] [Warning] feature_fraction is set=0.6042460007414772, colsample_bytree=1.0 will be ignored. Current value: feat

In [78]:
test_data_flit = test_data[result]
test_data_flit

,DEF_30_CNT_SOCIAL_CIRCLE,SK_ID_CURR_mean_instalment_paid_late_last_5,CNT_FAM_MEMBERS,external_sources_mean,credit_to_goods_ratio,phone_to_employ_ratio,pos_cash_completed_contracts,OBS_30_CNT_SOCIAL_CIRCLE,SK_ID_CURR_max_instalment_paid_late_last_5,external_sources_sum,...,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY,external_sources_nanmedian,payment_rate,SK_ID_CURR_mean_CNT_DRAWINGS_ATM_CURRENT,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH,credit_to_annuity_ratio,OCCUPATION_TYPE_mean_DAYS_EMPLOYED,SK_ID_CURR_max_CNT_PAYMENT_last_5,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN,previous_application_prev_was_refused
0,0.0,0.0,2.0,0.567263,1.2640,0.747102,2.0,0.0,0.0,1.701788,...,28477.113040,0.752614,0.036147,0.440091,-3158.810207,27.664697,-2479.794924,8.0,0.349544,0.0
1,0.0,0.2,2.0,0.429869,1.2376,1.059559,1.0,0.0,1.0,1.289607,...,31361.437181,0.432962,0.077973,0.440091,-3060.258241,12.824870,-1875.180147,12.0,0.430491,0.0
2,0.0,0.0,2.0,0.655389,1.0528,0.192014,3.0,0.0,0.0,1.310778,...,31361.437181,0.655389,0.105202,0.255556,-3060.258241,9.505482,-2057.260728,36.0,0.430491,0.0
3,0.0,0.0,4.0,0.549372,1.0000,0.967310,2.0,0.0,0.0,1.648115,...,28477.113040,0.525734,0.031123,0.045455,-3158.810207,32.130726,-1805.572161,24.0,0.349544,0.0
4,0.0,0.0,3.0,0.313916,1.0000,0.374715,1.0,0.0,0.0,0.627832,...,30976.229222,0.313916,0.051266,0.440091,-2870.514892,19.506034,-2479.794924,36.0,0.497756,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0.0,0.0,1.0,0.645800,1.5280,0.132327,1.0,1.0,0.0,1.291600,...,28477.113040,0.645800,0.042354,0.440091,-3158.810207,23.610610,-2479.794924,24.0,0.349544,0.0
48740,0.0,0.0,4.0,0.684596,1.2574,1.059559,3.0,2.0,0.0,0.684596,...,28777.713648,0.684596,0.051267,0.440091,-2711.316251,19.505570,-1805.572161,24.0,0.462886,0.0
48741,0.0,0.0,3.0,0.549995,1.0000,0.275930,2.0,0.0,0.0,1.649985,...,28477.113040,0.632770,0.105414,0.440091,-3158.810207,9.486380,-2479.794924,12.0,0.349544,0.0
48742,0.0,0.0,2.0,0.471416,1.0000,0.845112,1.0,0.0,0.0,1.414247,...,30976.229222,0.445701,0.055840,0.440091,-2870.514892,17.908309,-2928.782876,24.0,0.497756,0.0


In [83]:
y_pred_proba = model_flit.predict_proba(test_data_flit)[:, 1]
y_pred_proba = pd.DataFrame(y_pred_proba, columns=['TARGET'],index = test_ori['SK_ID_CURR'])
y_pred_proba.index.name = 'SK_ID_CURR'
y_pred_proba

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] lambda_l1 is set=22.70784666063976, reg_alpha=0.0 will be ignored. Current value: lambda_l1=22.70784666063976
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] lambda_l2 is set=40.733683783736836, reg_lambda=0.0 will be ignored. Current value: lambda_l2=40.733683783736836
[LightGBM] [Warning] bagging_fraction is set=0.8802727495826554, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8802727495826554
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_gain_to_split is set=11, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=11
[LightGBM] [Warning] feature_fraction is set=0.6042460007414772, colsample_bytree=1.0 will be ignored. Current value: feat

,TARGET
SK_ID_CURR,
100001,0.563214
100005,0.518885
100013,0.376073
100028,0.099654
100038,0.768396
...,...
456221,0.270075
456222,0.409492
456223,0.067081
